In [162]:
import geopandas as gpd
import pandas as pd
import json
import copy

In [215]:
state = 'MI'

In [225]:
pos = {
    'IA': {
        'name': 'Iowa',
        'scale': 2530,
        'translate': [80, 310],
        'arrow': 2
    },
    'PA': {
        'name': 'Pennsylvania',
        'scale': 4000,
        'translate': [-800, 380],
        'arrow': 2
    },
    'WI': {
        'name': 'Wisconsin',
        'scale': 2500,
        'translate': [-20,410],
        'arrow': 2
    },
    'MI': {
        'name': 'Michigan',
        'scale': 2070,
        'translate': [-90,370],
        'arrow': 2
    }
}

In [226]:
d1 = gpd.GeoDataFrame(pd.read_pickle('temp/windofchange.pkl'))
d1

,fips,county,state,geometry,winner,shifted,population,_merge,datensatz
0,13027,Brooks,GA,"MULTIPOLYGON (((-83.73616 31.03768, -83.57396 ...",Republicans,4.0,16245.0,both,NaN
1,31095,Jefferson,NE,"MULTIPOLYGON (((-97.36869 40.35039, -96.91606 ...",NaN,2.0,7054.0,both,NaN
2,31095,Jefferson,NE,"MULTIPOLYGON (((-97.36869 40.35039, -96.91606 ...",NaN,0.0,7054.0,both,NaN
3,51683,Manassas,VA,"MULTIPOLYGON (((-77.52666 38.73320, -77.49545 ...",NaN,-22.0,42696.0,both,NaN
4,56021,Laramie,WY,"MULTIPOLYGON (((-105.27824 41.65666, -104.6533...",NaN,-24.0,100984.0,both,NaN
...,...,...,...,...,...,...,...,...,...
3338,9007,NaN,NaN,None,Republicans,4.0,NaN,right_only,Nikolai
3339,9008,NaN,NaN,None,Democrats,-9.0,NaN,right_only,Nikolai
3340,9009,NaN,NaN,None,Republicans,4.0,NaN,right_only,Nikolai
3341,9010,NaN,NaN,None,Democrats,12.0,NaN,right_only,Nikolai


In [227]:
d2 = d1[d1.state == state]
d2

,fips,county,state,geometry,winner,shifted,population,_merge,datensatz
18,26105,Mason,MI,"MULTIPOLYGON (((-86.51470 44.05812, -86.42987 ...",NaN,16.0,29159.0,both,NaN
34,26007,Alpena,MI,"MULTIPOLYGON (((-83.88848 44.85663, -83.88122 ...",NaN,6.0,28904.0,both,NaN
76,26001,Alcona,MI,"MULTIPOLYGON (((-83.88848 44.85646, -83.31627 ...",NaN,-3.0,10489.0,both,NaN
188,26039,Crawford,MI,"MULTIPOLYGON (((-84.84731 44.85804, -84.37174 ...",NaN,6.0,13538.0,both,NaN
191,26143,Roscommon,MI,"MULTIPOLYGON (((-84.85093 44.51107, -84.37064 ...",NaN,4.0,23863.0,both,NaN
...,...,...,...,...,...,...,...,...,...
3069,26035,Clare,MI,"MULTIPOLYGON (((-85.08740 44.16424, -84.85170 ...",NaN,-15.0,31316.0,both,NaN
3189,26033,Chippewa,MI,"MULTIPOLYGON (((-85.23787 46.75570, -85.17304 ...",NaN,0.0,36264.0,both,NaN
3235,26159,Van Buren,MI,"MULTIPOLYGON (((-86.35622 42.25417, -86.28445 ...",NaN,10.0,75798.0,both,NaN
3238,26049,Genesee,MI,"MULTIPOLYGON (((-83.93207 43.22038, -83.69562 ...",NaN,6.0,401522.0,both,NaN


In [228]:
d2.to_file('temp/'+state+'.geojson', driver='GeoJSON')

In [229]:
!geo2topo temp/{state}.geojson>temp/{state}.topo.json

In [230]:
with open('temp/'+state+'.topo.json', 'r') as f:
    stateData = json.load(f)

In [231]:
with open('specs/arrow-mw.vg.json', 'r') as f:
    spec_mw = json.load(f)

In [232]:
spec_mw['data'][1]['values'] = stateData
spec_mw['data'][1]['format']['feature'] = state
spec_mw['data'][0]['transform'] = [
        {
          "type": "filter",
          "expr": "datum.properties.name === '"+pos[state]['name']+"'"
        }
      ]
spec_mw['projections'][0]['scale'] = pos[state]['scale']
spec_mw['projections'][0]['translate'] = pos[state]['translate']
spec_mw['signals'][0]['value'] = pos[state]['arrow']

In [233]:
with open('temp/specs/arrow-'+state+'-mw.vg.json', 'w') as f:
    json.dump(spec_mw, f, indent=4)

In [ ]:
spec_cw = copy.deepcopy(spec_mw)
spec_cw["width"] = 630
spec_cw["height"] = 500

spec_cw['projections'][0]['scale'] = pos[state]['scale']*cw_scale
spec_cw['projections'][0]['translate'] = [pos[state]['translate'][0]*cw_scale, pos[state]['translate'][1]*cw_scale]

with open('temp/specs/arrow-'+state+'-cw.vg.json', 'w') as f:
    json.dump(spec_cw, f, indent=4)
